In [1]:
import numpy as np

In [12]:
class Tokenizer(object):
    
    def get_words_probs(self, sentence):
        """获取单词分词结果与概率，具体方法已完成，在另一台电脑里，已字典树的形式实现"""
        words = ["经常", "经", "有", "有意见", "意见", "分歧", "见", "意", "见分歧", "分"]
        probs = [0.1, 0.05, 0.1, 0.1, 0.2, 0.2, 0.05, 0.05, 0.05, 0.1]
        return words, probs
    
    def get_words_lose_ln_probs(self, sentence):
        """获取初步分词结果与概率的-log()(实际代表数学公式的-ln())"""
        words, probs = self.get_words_probs(sentence)
        lose_ln_probs = -np.log(probs)
        lose_ln_probs = np.around(lose_ln_probs, decimals=2)
        # np.set_printoptions(precision=4)
        return np.array(words), lose_ln_probs
    
    def node_tree(self, sentence, words, lose_ln_probs):
        probs = [[100, None]]
        for n, char in enumerate(sentence, 1):
            probs.append([100 + probs[n-1][0], n-1])
            for word, prob in zip(words, lose_ln_probs):
                if len(word) <= n and word == sentence[n - len(word): n]:
                    new_prob = prob
                    before_n = n - len(word)
                    if before_n > 0:
                        new_prob += probs[before_n][0]
                    if new_prob < probs[n][0]: 
                        probs[n] = [new_prob, before_n]
        return probs
    
    def _cut(self, sentence, sentence_probs):
        next_n = -1
        words = []
        while True:
            words.append(sentence[sentence_probs[next_n][1]:])
            sentence = sentence[:sentence_probs[next_n][1]]
            if sentence_probs[next_n][1] == 0:
                break
            next_n = sentence_probs[next_n][1]
        return words[::-1]
    
    def cut(self, sentence):
        words, lose_ln_probs = self.get_words_lose_ln_probs(sentence)
        # print(words)
        # print(lose_ln_probs)
        sentence_probs = self.node_tree(sentence, words, lose_ln_probs)
        # print(sentence_probs)
        words = self._cut(sentence, sentence_probs)
        # print(words)
        return words


In [13]:
sentence = '经常有意见分歧'
t = Tokenizer()
t.cut(sentence)

['经常', '有意见', '分歧']

In [14]:
type(-np.log([0.1, 0.2]))

numpy.ndarray